In [ ]:
import itertools
import importlib
from pathlib import Path

import pandas as pd
import numpy as np

from bokeh.io import output_notebook
output_notebook()

In [ ]:
from trafficdata import parser 

importlib.reload(parser)

minutes_to_bin = 5

def week_bin(buf: int) -> int:
    return (parser.Column.weekday(buf) * 24 + parser.Column.hour(buf)) * 12 + (parser.Column.minute(buf) // 5)

def day_bin(buf: int) -> int:
    return parser.Column.hour(buf) * 12 + (parser.Column.minute(buf) // minutes_to_bin)

def bytes_linear(buf: int) -> int:
    return 1 << parser.Column.log_bytes(buf)


datadir = Path("data")
days = list(sorted(datadir.iterdir()))
print(days)
columns = parser.read_columns(
    [d for d in days if "2022" in d.name], 
    [
        ("weekday", parser.Column.weekday),
        ("bin", week_bin),
        ("x", day_bin),
        ("y", bytes_linear),
    ],
    file_idx_col="week"
)
print("finished read")
print(columns)
d = pd.DataFrame(data=columns)

In [ ]:
sums = d.groupby(["week", "weekday", "bin", "x"]).sum()
sums
avgs = sums.groupby(["weekday", "bin", "x"]).mean()

In [ ]:
from bokeh.palettes import Dark2_5 as palette
from bokeh.plotting import figure, show, output_file, save

bin_labels = {
    300: ""
}
for bin in range((24 * 60) // minutes_to_bin):
    hour = (bin * minutes_to_bin) // 60
    minute = (bin * minutes_to_bin) % 60
    posmod_hour = ((hour + 11) % 12) + 1
    bin_labels[bin] = f"{posmod_hour}:{minute:02} {'am' if hour < 12 else 'pm'}"

p = figure(width=900, height=400) # , y_axis_type="log")
p.xaxis.major_label_overrides = bin_labels

days = {
    "Monday": "#dfdf00",
    "Tuesday": "#ffc0cb",
    "Wednesday": "#008000",
    "Thursday": "#ffa500",
    "Friday": "#00bfff",
    "Saturday": "#800080",
    "Sunday": "#ff0000"
}

for weekday, (name, color) in enumerate(days.items()):
    day_avgs = avgs.query(f"weekday == {weekday}")
    xs = day_avgs.axes[0].get_level_values(2)
    ((_, ys), ) = day_avgs.items()
    # ys = np.exp(ys / 100.0)
    p.line(x=xs, y=ys, line_width=2, color=color, legend_label=name)
p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)
if False:
    output_file("traffic.html", title="Home Network Traffic Graph")
    save(p)